#0. Prerequisites

In [1]:
!pip install amadeus

  Preparing metadata (setup.py) ... done
  Created wheel for amadeus: filename=amadeus-12.0.0-py2.py3-none-any.whl size=67478 sha256=4f62b981a062d275cb98df97edf18db4d96e9a72d3026b43020db9dd0d100364
  Stored in directory: /root/.cache/pip/wheels/78/e5/a1/54b22441ac55fb018faea6f8bfd364347b945bf14f1499f257
Successfully built amadeus


In [2]:
!pip install openai
!pip install python-dotenv

In [42]:
pip install dateparser

In [43]:
import json
import re
from datetime import datetime
import dateparser

from openai import OpenAI
from amadeus import Client, ResponseError

In [44]:
AMADEUS_CLIENT_ID = "Kx9ymBFmpNUweZbSIyNaZePHPhcLgE3G" # this line is invisible!
AMADEUS_CLIENT_SECRET = "iL7TxFUK8oqUJBOu" # this one too!!

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
from dotenv import load_dotenv
dotenv_path = '/content/drive/MyDrive/8- CMPE 297 Natural Language Processing/HW1/OPENAI_API_KEY.env'  # Specify the path to the .env file
load_dotenv(dotenv_path)

True

In [47]:
amadeus = Client(
    client_id= AMADEUS_CLIENT_ID,
    client_secret= AMADEUS_CLIENT_SECRET
)
client = OpenAI()

In [48]:
AIRLINES = {
    # 🇺🇸 United States
    "AA": "American Airlines",
    "DL": "Delta Air Lines",
    "UA": "United Airlines",
    "WN": "Southwest Airlines",
    "B6": "JetBlue Airways",
    "AS": "Alaska Airlines",
    "NK": "Spirit Airlines",
    "F9": "Frontier Airlines",
    "G4": "Allegiant Air",
    "HA": "Hawaiian Airlines",

    # 🇨🇦 Canada
    "AC": "Air Canada",
    "WS": "WestJet",
    "TS": "Air Transat",

    # 🇲🇽 Mexico
    "AM": "Aeroméxico",
    "VB": "Viva Aerobus",
    "Y4": "Volaris",

    # 🇬🇧 United Kingdom & Ireland
    "BA": "British Airways",
    "EI": "Aer Lingus",
    "U2": "easyJet",
    "FR": "Ryanair",
    "LS": "Jet2",

    # 🇪🇺 Europe (Major)
    "LH": "Lufthansa",
    "AF": "Air France",
    "KL": "KLM Royal Dutch Airlines",
    "IB": "Iberia",
    "AZ": "ITA Airways",
    "LX": "SWISS",
    "OS": "Austrian Airlines",
    "SN": "Brussels Airlines",
    "SK": "Scandinavian Airlines (SAS)",
    "TP": "TAP Air Portugal",
    "AY": "Finnair",
    "LO": "LOT Polish Airlines",
    "OK": "Czech Airlines",

    # 🇪🇺 Low-cost Europe
    "VY": "Vueling",
    "W6": "Wizz Air",
    "EW": "Eurowings",
    "DY": "Norwegian Air Shuttle",
    "PC": "Pegasus Airlines",

    # 🇹🇷
    "TK": "Turkish Airlines",

    # 🇮🇱
    "LY": "El Al Israel Airlines",

    # 🇷🇺 (may appear in data)
    "SU": "Aeroflot",

    # 🇯🇵 Japan
    "JL": "Japan Airlines",
    "NH": "All Nippon Airways (ANA)",
    "MM": "Peach Aviation",
    "GK": "Jetstar Japan",

    # 🇨🇳 China
    "CA": "Air China",
    "MU": "China Eastern Airlines",
    "CZ": "China Southern Airlines",
    "HU": "Hainan Airlines",

    # 🇰🇷 South Korea
    "KE": "Korean Air",
    "OZ": "Asiana Airlines",
    "TW": "T’way Air",
    "7C": "Jeju Air",

    # 🇸🇬
    "SQ": "Singapore Airlines",
    "TR": "Scoot",

    # 🇭🇰
    "CX": "Cathay Pacific",
    "HX": "Hong Kong Airlines",

    # 🇹🇭
    "TG": "Thai Airways",
    "FD": "Thai AirAsia",

    # 🇮🇳 India
    "AI": "Air India",
    "UK": "Vistara",
    "6E": "IndiGo",
    "SG": "SpiceJet",
    "G8": "Go First",

    # 🇦🇺 Australia & NZ
    "QF": "Qantas",
    "VA": "Virgin Australia",
    "JQ": "Jetstar Airways",
    "NZ": "Air New Zealand",

    # 🌍 Middle East
    "EK": "Emirates",
    "QR": "Qatar Airways",
    "EY": "Etihad Airways",
    "SV": "Saudia",
    "GF": "Gulf Air",
    "WY": "Oman Air",
    "KU": "Kuwait Airways",

    # 🌍 Africa
    "ET": "Ethiopian Airlines",
    "MS": "EgyptAir",
    "AT": "Royal Air Maroc",
    "SA": "South African Airways",
    "KQ": "Kenya Airways",

    # 🌍 South America
    "LA": "LATAM Airlines",
    "AV": "Avianca",
    "CM": "Copa Airlines",
    "G3": "GOL Linhas Aéreas",
    "AD": "Azul Brazilian Airlines",
    "AR": "Aerolineas Argentinas",

    # 🌍 Budget / Global
    "AK": "AirAsia",
    "D7": "AirAsia X",
    "Z2": "Philippines AirAsia",
    "UO": "Hong Kong Express",
}

#1. Helpers

In [60]:
def parse_date(text):
    date = dateparser.parse(text, settings={"PREFER_DATES_FROM": "future"})
    return date.strftime("%Y-%m-%d") if date else None

def format_duration(iso_duration):
    match = re.search(r'PT(\d+)H(\d+)M', iso_duration)
    if not match:
        return iso_duration
    h, m = match.groups()
    return f"{h} hours {m} minutes"

def city_to_iata(city):
    try:
        res = amadeus.reference_data.locations.get(
            keyword=city,
            subType="AIRPORT,CITY"
        )
        return res.data[0]["iataCode"]
    except:
        return None

def format_flights(flights, origin_city, destination_city, date):
    if not flights:
        return f"No flights found from {origin_city} to {destination_city} on {date}."
    response = f"✈️ Flights from {origin_city} to {destination_city} on {date}:\n\n"
    for f in flights:
        response += f"• {f['airline']} — ${f['price']} — {f['duration']}\n"
    return response

#2- Search Flights

In [65]:
def search_flights(origin_iata, destination_iata, date):
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin_iata,
            destinationLocationCode=destination_iata,
            departureDate=date,
            adults=1,
            max=5
        )
        flights = []
        for offer in response.data:
            segment = offer["itineraries"][0]["segments"][0]
            code = segment["carrierCode"]
            airline = AIRLINES.get(code, f"Airline ({code})")
            duration = format_duration(offer["itineraries"][0]["duration"])
            price = float(offer["price"]["total"])
            flights.append({"airline": airline, "price": price, "duration": duration})
        return sorted(flights, key=lambda x: x["price"])
    except ResponseError:
        return []

#3- Parse Trip

In [67]:
def parse_trip(user_input):
    today = datetime.today().strftime("%Y-%m-%d")
    prompt = f"""
You are a travel assistant.

Extract the origin city, destination city, and departure date from the user request.
Return only JSON. Use YYYY-MM-DD for dates. If date is not specified, return null.

Today is {today}.

Example JSON:
{{
  "origin_city": "Los Angeles",
  "destination_city": "Dallas",
  "date": "2026-02-02"
}}

User input: "{user_input}"
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    trip = json.loads(response.choices[0].message.content)
    if trip.get("date"):
        trip["date"] = parse_date(trip["date"])
    return trip

#4- Chat Loop

In [69]:
print("✈️ Travel Chatbot (type 'exit' to quit)\n")

trip_memory = {
    "origin_city": None,
    "destination_city": None,
    "date": None,
    "last_flights": None
}

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    try:
        trip_info = parse_trip(user_input)

        # Update memory
        for key in ["origin_city", "destination_city", "date"]:
            if trip_info.get(key):
                trip_memory[key] = trip_info[key]

        # Check for missing info
        missing_info = [k for k in ["origin_city", "destination_city", "date"] if not trip_memory.get(k)]
        if missing_info:
            response = f"I need more info to search for flights: {', '.join(missing_info)}."
        else:
            flights = search_flights(
                city_to_iata(trip_memory["origin_city"]),
                city_to_iata(trip_memory["destination_city"]),
                trip_memory["date"]
            )
            trip_memory["last_flights"] = flights
            response = format_flights(flights, trip_memory["origin_city"], trip_memory["destination_city"], trip_memory["date"])

    except Exception as e:
        response = f"Error: {e}"

    print("\nBot:", response, "\n")

✈️ Travel Chatbot (type 'exit' to quit)

You: find cheapest flight from los angeles to dallas next monday

Bot: ✈️ Flights from Los Angeles to Dallas on 2026-02-06:

• Frontier Airlines — $105.34 — 3 hours 2 minutes
• Spirit Airlines — $105.34 — 3 hours 4 minutes
• Spirit Airlines — $105.34 — 3 hours 7 minutes
• Spirit Airlines — $140.42 — 6 hours 50 minutes
• Spirit Airlines — $140.42 — 8 hours 13 minutes
 

You: exit
